# 第三章：数字日期和时间
在Python中执行整数和浮点数的数学运算时很简单。
但是，如果你需要执行分数、数组或者是日期和时间的运算的话，就得注意更多细节了

## 3.1 数字的四舍五入
* 问题

对浮点数执行指定精度的舍入运算

* 解决方案

对于简单的舍入运算，使用内置的 `round(value, ndigits)` 函数即可。

In [8]:
round(1.23, 1)  # 保留一位小数

1.2

In [2]:
round(1.27, 1)  # 保留一位小数

1.3

In [3]:
round(-1.27, 1)

-1.3

从上面例子中可以看出，round函数的规则并不是四舍五入，而是离哪个数近，就近似于它

In [4]:
round(1.234312312, 3)  # 保留三位有效数字

1.234

当一个值刚好在两个边界的中间的时候， round 函数返回离它最近的偶数。 也就是说，对1.5或者2.5的舍入运算都会得到2。

In [10]:
round(1.5)

2

In [7]:
round(2.5)

2

In [11]:
round(1.25, 1)  # [1.2 1.3] -> 2是最近的偶数，所以是1.25

1.2

In [12]:
round(1.35, 1)  # [1.3 1.4] -> 离0.35最近的偶数是0.4


1.4

传给 `round()` 函数的 `ndigits` 参数可以是负数，这种情况下， 舍入运算会作用在十位、百位、千位等上面。比如：

In [17]:
a = 1627731
round(a, -1)

1627730

In [19]:
round(a, -2)

1627700

In [15]:
round(a, -3)

1628000

### 讨论
不要将舍入和格式化输出搞混淆了。
如果你的目的只是简单的输出一定宽度的数，你不需要使用 round() 函数。
而仅仅只需要在格式化的时候指定精度即可。比如：

In [23]:
x = 1.23456
format(x, '0.2f')

'1.23'

In [21]:
'value is {:0.3f}'.format(x)

'value is 1.235'

同样，不要试着去舍入浮点值来”修正”表面上看起来正确的问题。

In [25]:
a = 2.1
b = 4.2
c = a + b
print('c is {}'.format(c))

c is 6.300000000000001


In [26]:
c = round(c, 1)
print('after round, c is {}'.format(c))

after round, c is 6.3


对于大多数使用到浮点的程序，没有必要也不推荐这样做。 尽管在计算的时候会有一点点小的误差，但是这些小的误差是能被理解与容忍的。

如果不能允许这样的小误差(比如涉及到金融领域)，那么就得考虑使用 `decimal` 模块了，下一节我们会详细讨论。

## 3.2 执行精确的浮点数运算

### 问题
你需要对浮点数执行**精确的计算操作**，并且不希望有任何小误差的出现。

### 解决方案
浮点数的一个普遍问题是它们并不能精确的表示十进制数。 并且，即使是最简单的数学运算也会产生小的误差。

In [27]:
a = 4.2
b = 2.1
c = a + b

c == 6.3

False


这些错误是由底层CPU和IEEE 754标准通过自己的浮点单位去执行算术时的特征。

由于Python的浮点数据类型使用底层表示存储数据，因此你没办法去避免这样的误差。

如果你想更加精确，可以使用 `decimal` 模块，但是会造成程序执行时间变慢，造成性能损耗

In [30]:
from decimal import Decimal
a = Decimal('4.2')
b = Decimal('2.1')
c = a + b

c == Decimal('6.3')

True

初看起来，上面的代码好像有点奇怪，我们用**字符串来表示数字**。
然而， `Decimal` 对象会像普通浮点数一样的工作(支持所有的常用数学运算)。
如果你打印它们或者在字符串格式化函数中使用它们，看起来跟普通数字没什么两样。

`decimal` 模块的一个主要特征是允许你控制计算的每一方面，包括数字位数和四舍五入运算。 为了这样做，你先得创建一个**本地上下文`localcontext`**并更改它的设置，比如：

In [31]:
from decimal import localcontext
a = Decimal('1.3')
b = Decimal('1.7')
print(a / b)

0.7647058823529411764705882353


In [32]:
with localcontext() as ctx:
    ctx.prec = 3
    print(a / b)

0.765


In [33]:
with localcontext() as ctx:
    ctx.prec = 50
    print(a / b)

0.76470588235294117647058823529411764705882352941176


### 讨论
`decimal` 模块实现了IBM的”通用小数运算规范”。

Python新手会倾向于使用 `decimal` 模块来处理浮点数的精确运算。
然而，先理解你的应用程序目的是非常重要的。如果你是在做科学计算或工程领域的计算、电脑绘图，或者是科学领域的大多数运算，那么使用普通的浮点类型即可。

其中一个原因是，**在真实世界中很少会要求精确到普通浮点数能提供的17位精度**。因此，计算过程中的那么一点点的误差是被允许的。
第二点就是，**原生的浮点数计算要快的多** --当执行大量运算的时候速度也是非常重要的。

即便如此，你却不能完全忽略误差。数学家花了大量时间去研究各类算法，有些处理误差会比其他方法更好。也需要注意减法删除以及大数和小数的加法运算所带来的影响。

In [34]:
nums = [1.23e+18, 1, -1.23e+18]
sum(nums) # Notice how 1 disappears

0.0

In [35]:
# 上面的错误可以利用 math.fsum() 所提供的更精确计算能力来解决：
import math
math.fsum(nums)

1.0

然而，对于其他的算法，你应该仔细研究它并理解它的误差产生来源。

总的来说， `decimal` 模块主要用在涉及到金融的领域。在这类程序中，哪怕是一点小小的误差在计算过程中都不被允许的。因此，`decimal` 模块为解决这类问题提供了方法。

Python处理数据库中的金融数据时，也通常会遇到 `Decimal` 对象。

## 3.3 数字的格式化输出
* 问题

将数字格式化后输出，并控制数字的位数、对齐、千位分隔符和其他的细节。

* 解决方案

格式化输出单个数字的时候，可以使用内置的 `format()` 函数

In [1]:
x = 1234.5678
# Two decimal places of accuracy
format(x, '0.2f')

'1234.57'

In [2]:
# Right justified in 10 chars, one-digit accuracy
format(x, '>10.1f')

'    1234.6'

In [3]:
# Left justified
format(x, '<10.1f')

'1234.6    '

In [4]:
# Centered
format(x, '^10.1f')

'  1234.6  '

In [5]:
# Inclusion of thousands separator
format(x, ',')

'1,234.5678'

In [6]:
format(x, '0,.1f')

'1,234.6'

In [7]:
format(x, ',.1f')


'1,234.6'

如果你想使用指数记法，将f改成e或者E(取决于指数输出的大小写形式)。比如：

In [8]:
format(x, 'e')

'1.234568e+03'

In [9]:
format(x, '0.2E')

'1.23E+03'

同时指定宽度和精度的一般形式是 `'[<>^]?width[,]?(.digits)?'` ， 其中 `width` 和 `digits` 为整数，`？` 代表可选部分。同样的格式也被用在字符串的 `format()` 方法中。比如：

In [10]:
'The value is {:0,.2f}'.format(x)

'The value is 1,234.57'

* 讨论

数字格式化输出通常是比较简单的。上面演示的技术同时适用于**浮点数**和 `decimal` 模块中的 `Decimal` 数字对象。

当指定数字的位数后，结果值会根据 round() 函数同样的规则进行四舍五入后返回。

In [11]:
x

1234.5678

In [12]:
format(x, '0.1f')

'1234.6'

In [13]:
format(-x, '0.1f')

'-1234.6'

包含千位符的格式化跟本地化没有关系。 如果你需要根据地区来显示千位符，你需要自己去调查下 `locale` 模块中的函数了。 你同样也可以使用字符串的 `translate()` 方法来交换千位符。比如：

In [14]:
swap_separators = { ord('.'):',', ord(','):'.' }
format(x, ',').translate(swap_separators)

'1.234,5678'

还可以使用 `%` 来格式化数字的

In [ ]:
'%0.2f' % x

In [23]:
'%10.2f' % x

'   1234.57'

In [17]:
'%-10.1f' % x

'1234.6    '

In [21]:
'%^10.1f' % x

ValueError: unsupported format character '^' (0x5e) at index 1

In [18]:
'%0.2E' % x

'1.23E+03'

In [19]:
'%e' % x

'1.234568e+03'

In [24]:
'%,' % x

ValueError: unsupported format character ',' (0x2c) at index 1

In [20]:
'%0,.1f' % x  # 无法添加千位符号

ValueError: unsupported format character ',' (0x2c) at index 2

这种格式化方法也是可行的，不过比更加推荐使用 `format()`。 比如，在使用 `%` 操作符格式化数字的时候，一些特性(添加千位符)并不能被支持。

## 3.4 二八十六进制整数
* 问题

你需要转换或者输出使用二进制，八进制或十六进制表示的整数。

* 解决方案

为了将整数转换为二进制、八进制或十六进制的文本串， 可以分别使用 `bin()` , `oct()` 或 `hex()` 函数：

In [25]:
x = 1234
bin(x)

'0b10011010010'

In [26]:
oct(x)

'0o2322'

In [27]:
hex(x)

'0x4d2'

另外，如果你不想输出 `0b`, `0o` 或者 `0x` 的前缀的话，可以使用 format() 函数。比如：

In [28]:
format(x, 'b')

'10011010010'

In [29]:
format(x, 'o')

'2322'

In [30]:
format(x, 'x')

'4d2'

整数是有符号的，所以如果你在处理负数的话，输出结果会包含一个负号。

In [31]:
x = -1234
format(x, 'b')

'-10011010010'

In [33]:
format(x, 'x')

'-4d2'

如果你想产生一个无符号值，你需要增加一个指示最大位长度的值。比如为了显示32位的值

In [34]:
x = -1234
format(2**32 + x, 'b')

'11111111111111111111101100101110'

In [35]:
format(2**32 + x, 'x')

'fffffb2e'

为了以不同的进制转换整数字符串，简单的使用带有进制的 `int()` 函数即可：

In [36]:
int('4d2', 16)

1234

In [37]:
int('10011010010', 2)

1234

* 讨论

大多数情况下处理二进制、八进制和十六进制整数是很简单的。只要记住这些转换属于整数和其对应的文本表示之间的转换即可。永远只有一种整数类型。

最后，使用八进制的程序员有一点需要注意下。Python指定八进制数的语法跟其他语言稍有不同。比如，如果你像下面这样指定八进制，会出现语法错误：

In [40]:
import os
os.chmod('script.py', 0755)

SyntaxError: invalid token (<ipython-input-40-7619b0e4e554>, line 2)

In [41]:
# 需确保八进制数的前缀是 0o ，就像下面这样：
os.chmod('script.py', 0o755)

## 3.5 字节到大整数的打包与解包
* 问题

将一个 **字节字符串** 解压成一个整数；
将一个大整数转换一个字节字符串。

* 解决方案

假设你的程序需要处理一个拥有128位长的16个元素的字节字符串。

In [1]:
data = b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

为了将bytes解析为整数，使用 `int.from_bytes()` 方法，并指定字节顺序：

In [2]:
len(data)

16

In [3]:
int.from_bytes(data, 'little')

69120565665751139577663547927094891008

In [4]:
int.from_bytes(data, 'big')

94522842520747284487117727783387188

为了将一个大整数转换为一个字节字符串，使用 `int.to_bytes()` 方法，并像下面这样指定字节数和字节顺序：

In [5]:
x = 94522842520747284487117727783387188
x.to_bytes(16, 'big')

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [7]:
x.to_bytes(16, 'little')

b'4\x00#\x00\x01\xef\xcd\x00\xab\x90x\x00V4\x12\x00'

* 讨论

大整数和字节字符串之间的转换操作并不常见。 然而，在一些应用领域有时候也会出现，比如密码学或者网络。例如，IPv6网络地址使用一个128位的整数表示。 如果你要从一个数据记录中提取这样的值的时候，你就会面对这样的问题。

作为一种替代方案，你可能想使用 `struct` 模块来解压字节。 这样也行得通，不过利用 `struct` 模块来解压，对于整数的大小是有限制的。 因此，你可能想解压多个字节串并将结果合并为最终的结果，就像下面这样：

In [8]:
data

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [10]:
import struct

hi, lo = struct.unpack('>QQ', data)
(hi << 64) + lo

94522842520747284487117727783387188

字节顺序规则(little或big)仅仅指定了构建整数时的字节的低位高位排列方式。 我们从下面构造的16进制数的表示中可以很容易的看出来：

In [11]:
x = 0x01020304
x.to_bytes(4, 'big')

b'\x01\x02\x03\x04'

In [12]:
x.to_bytes(4, 'little')

b'\x04\x03\x02\x01'

如果你试着将一个整数打包为字节字符串，程序会报错。 如果需要的话，你可以使用 `int.bit_length()` 方法来决定需要多少字节位来存储这个值。

In [15]:
x = 523 ** 23
x

335381300113661875107536852714019056160355655333978849017944067

In [16]:
x.to_bytes(16, 'little')

OverflowError: int too big to convert

In [17]:
x.bit_length()

208

In [18]:
nbytes, rem = divmod(x.bit_length(), 8)
if rem:
    nbytes += 1

x.to_bytes(nbytes, 'little')

b'\x03X\xf1\x82iT\x96\xac\xc7c\x16\xf3\xb9\xcf\x18\xee\xec\x91\xd1\x98\xa2\xc8\xd9R\xb5\xd0'

## 3.6 复数的数学运算

* 问题

你写的最新的网络认证方案代码遇到了一个难题，并且你唯一的解决办法就是使用复数空间。 再或者是你仅仅需要使用复数来执行一些计算操作。

* 解决方案

复数可以用使用函数 `complex(real, imag)` 或者是带有后缀 `j` 的浮点数来指定。

In [19]:
a = complex(2, 4)
b = 3 - 5j
print(a)
print(b)

(2+4j)
(3-5j)


In [20]:
# 获取对应的实部、虚部和共轭复数
print(a.real)
print(a.imag)
print(a.conjugate())

2.0
4.0
(2-4j)


In [21]:
# 常见的数学运算
print(a + b)
print(a * b)
print(a / b)
print(abs(a))

(5-1j)
(26+2j)
(-0.4117647058823529+0.6470588235294118j)
4.47213595499958


如果要执行其他的复数函数比如正弦、余弦或平方根，使用 `cmath` 模块

In [22]:
import cmath

cmath.sin(a)

(24.83130584894638-11.356612711218173j)

In [23]:
cmath.cos(a)

(-11.36423470640106-24.814651485634183j)

In [24]:
cmath.exp(a)

(-4.829809383269385-5.5920560936409816j)

* 讨论

Python中大部分与数学相关的模块都能处理复数。 比如如果你使用 `numpy` ，可以很容易的构造一个复数数组并在这个数组上执行各种操作：

In [26]:
import numpy as np

a = np.array([2+3j, 4+5j, 6-7j, 8+9j])
a

array([2.+3.j, 4.+5.j, 6.-7.j, 8.+9.j])

In [27]:
a + 2

array([ 4.+3.j,  6.+5.j,  8.-7.j, 10.+9.j])

In [28]:
np.sin(a)

array([   9.15449915  -4.16890696j,  -56.16227422 -48.50245524j,
       -153.20827755-526.47684926j, 4008.42651446-589.49948373j])

Python的标准数学函数确实情况下并不能产生复数值，因此你的代码中不可能会出现复数返回值。比如：

In [29]:
import math
math.sqrt(-1)

ValueError: math domain error

如果你想生成一个复数返回结果，你必须显示的使用 `cmath` 模块，或者在某个支持复数的库中声明复数类型的使用。

In [30]:
import cmath
cmath.sqrt(-1)

1j

## 3.7 无穷大与NaN

* 问题

创建或测试正无穷、负无穷或NaN(非数字)的浮点数。

* 解决方案

Python并没有特殊的语法来表示这些特殊的浮点值，但是可以使用 `float()` 来创建它们。

In [31]:
a = float('inf')
b = float('-inf')
c = float('nan')

In [32]:
a

inf

In [33]:
b

-inf

In [34]:
c

nan

为了测试这些值的存在，使用 `math.isinf()` 和 `math.isnan()` 函数。比如：

In [35]:
math.isinf(a)

True

In [37]:
math.isinf(-a)

True

In [36]:
math.isnan(c)

True

* 讨论

想了解更多这些特殊浮点值的信息，可以参考IEEE 754规范。 然而，也有一些地方需要你特别注意，特别是跟比较和操作符相关的时候。

In [38]:
# 无穷大数 inf 在执行数学计算的时候会传播
a = float('inf')
a + 45

inf

In [39]:
a * 10

inf

In [40]:
10 / a

0.0

In [41]:
# 有些操作未定义的并会返回一个NaN结果。比如：
a / a

nan

In [42]:
a + b

nan

In [43]:
# NaN值会在所有操作中传播，而不会产生异常。
c = float('nan')
c + 23

nan

In [47]:
c / 2

nan

In [45]:
c * 2

nan

In [46]:
math.sqrt(c)

nan

In [48]:
# NaN值的一个特别的地方时它们之间的比较操作总是返回False。
c = float('nan')
d = float('nan')
c == d

False

In [49]:
c is d

False

由于这个原因，测试`NaN值` 的唯一安全的方法就是使用 `math.isnan()`。

In [50]:
math.isnan(c)

True


有时候程序员想改变Python默认行为，在返回无穷大或NaN结果的操作中抛出异常。

`fpectl` 模块可以用来改变这种行为，但是它在标准的Python构建中并没有被启用，它是平台相关的， 并且针对的是专家级程序员。可以参考在线的Python文档获取更多的细节。

## 3.8 分数运算
* 问题

你进入时间机器，突然发现你正在做小学家庭作业，并涉及到分数计算问题。 或者你可能需要写代码去计算在你的木工工厂中的测量值。

* 解决方案

`fractions` 模块可以被用来执行包含分数的数学运算。

In [51]:
from fractions import Fraction
a = Fraction(5, 4)
b = Fraction(7, 16)
print(a + b)
print(a * b)

27/16
35/64


In [53]:
# Getting numerator/denominator
c = a * b
print(c.numerator)
print(c.denominator)

35
64


In [54]:
# Converting to a float
float(c)

0.546875

In [55]:
# Limiting the denominator of a value
print(c.limit_denominator(8))

4/7


In [56]:
# Converting a float to a fraction
x = 3.75
y = Fraction(*x.as_integer_ratio())
y

Fraction(15, 4)

* 讨论

在大多数程序中一般不会出现分数的计算问题，但是有时候还是需要用到的。

比如，在一个允许接受分数形式并以分数形式执行运算的程序中，直接使用分数可以减少手动转换为小数或浮点数的工作。